In [1]:
import pandas as pd
from collections import Counter
import ast
import numpy as np
import matplotlib.pyplot as plt
import datetime
# from skimage.io import imread
import os
import statistics
from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier

In [2]:
train_data = pd.read_csv('final_train_data.csv')

In [4]:
train_data=train_data[train_data['Pleural Effusion'].notna()]

In [5]:
train_data

,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,...,corner_193,corner_194,corner_195,corner_196,corner_197,corner_198,corner_199,Right_Lung_Intensity,Left_Lung_Intensity,Asymmetry
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,...,0,0,0,0,0,0,0,110.128293,96.570501,0.596201
5,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Frontal,PA,1.0,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,80.388532,75.504396,0.270156
6,CheXpert-v1.0-small/train/patient00004/study1/...,Female,20,Lateral,NaN,1.0,0.0,NaN,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
7,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Frontal,PA,1.0,NaN,0.0,NaN,NaN,...,0,0,0,0,0,0,0,70.260511,79.038964,0.225228
8,CheXpert-v1.0-small/train/patient00005/study1/...,Male,33,Lateral,NaN,1.0,NaN,0.0,NaN,NaN,...,0,0,0,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223403,CheXpert-v1.0-small/train/patient64533/study1/...,Male,75,Frontal,AP,NaN,NaN,1.0,NaN,NaN,...,0,0,0,0,0,0,0,95.301345,94.698435,0.285247
223405,CheXpert-v1.0-small/train/patient64534/study1/...,Male,63,Frontal,AP,NaN,NaN,NaN,1.0,NaN,...,0,0,0,0,0,0,0,64.932217,61.532157,0.267634
223407,CheXpert-v1.0-small/train/patient64536/study2/...,Female,61,Frontal,AP,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,143.463986,76.111103,0.727078
223409,CheXpert-v1.0-small/train/patient64537/study2/...,Male,59,Frontal,AP,NaN,NaN,NaN,-1.0,NaN,...,0,0,0,0,0,0,0,62.346834,91.663944,0.368473


In [6]:
# One-hot encode the categorical columns
train_data = pd.get_dummies(train_data, columns=['Sex'])
train_data = pd.get_dummies(train_data, columns=['Frontal/Lateral'])
train_data = pd.get_dummies(train_data, columns=['AP/PA'])

In [7]:
train_data

,Path,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,...,Asymmetry,Sex_Female,Sex_Male,Sex_Unknown,Frontal/Lateral_Frontal,Frontal/Lateral_Lateral,AP/PA_AP,AP/PA_LL,AP/PA_PA,AP/PA_RL
1,CheXpert-v1.0-small/train/patient00002/study2/...,87,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,...,0.596201,1,0,0,1,0,1,0,0,0
5,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,0.270156,1,0,0,1,0,0,0,1,0
6,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,1,0,0,0,1,0,0,0,0
7,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,0.225228,0,1,0,1,0,0,0,1,0
8,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223403,CheXpert-v1.0-small/train/patient64533/study1/...,75,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,0.285247,0,1,0,1,0,1,0,0,0
223405,CheXpert-v1.0-small/train/patient64534/study1/...,63,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.267634,0,1,0,1,0,1,0,0,0
223407,CheXpert-v1.0-small/train/patient64536/study2/...,61,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.727078,1,0,0,1,0,1,0,0,0
223409,CheXpert-v1.0-small/train/patient64537/study2/...,59,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,...,0.368473,0,1,0,1,0,1,0,0,0


In [8]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

def prepare_data(train_data, feature_cols, test_size=0.2, random_state=42, 
                pca=False, pca_components=0.95, impute_strategy='mean'):
    """
    Prepares data for machine learning tasks.
    
    Parameters:
    - train_data: DataFrame containing features and target
    - feature_cols: List of feature column names to use
    - test_size: Proportion of data for testing (default 0.2)
    - random_state: Random seed for reproducibility
    - pca: Whether to apply PCA (default False)
    - pca_components: Number of components or variance to retain
    - impute_strategy: Strategy for handling missing values ('mean', 'median', etc.)
    
    Returns:
    - X_train, X_test: Prepared feature arrays
    - y_train, y_test: Encoded target arrays
    - encoder: Fitted LabelEncoder (for inverse transform if needed)
    - imputer: Fitted imputer (for new data)
    - scaler: Fitted scaler (if PCA=True)
    - pca: Fitted PCA object (if PCA=True)
    """
    
    # Validate inputs
    if not isinstance(train_data, pd.DataFrame):
        raise ValueError("train_data must be a pandas DataFrame")
    if not all(col in train_data.columns for col in feature_cols):
        raise ValueError("Some feature_cols not found in train_data")
    
    # Remove rows with missing target values
    train_data = train_data.dropna(subset=['Pleural Effusion'])
    
    # Split features and target
    X = train_data[feature_cols].copy()  # Explicit copy to avoid SettingWithCopyWarning
    y = train_data['Pleural Effusion']
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )
    
    # Encode target variable
    encoder = LabelEncoder()
    y_train_encoded = encoder.fit_transform(y_train)
    y_test_encoded = encoder.transform(y_test)
    
    # Handle missing values
    imputer = SimpleImputer(strategy=impute_strategy)
    X_train_imputed = imputer.fit_transform(X_train)
    X_test_imputed = imputer.transform(X_test)
    
    # Initialize return objects
    scaler, pca_obj = None, None
    
    if pca:
        # Standardize features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        X_test_scaled = scaler.transform(X_test_imputed)
        
        # Apply PCA
        pca_obj = PCA(n_components=pca_components)
        X_train_pca = pca_obj.fit_transform(X_train_scaled)
        X_test_pca = pca_obj.transform(X_test_scaled)
        
        print(f"PCA: Retained {pca_obj.n_components_} components explaining "
              f"{100*pca_obj.explained_variance_ratio_.sum():.1f}% variance")
        
        return (X_train_pca, X_test_pca, y_train_encoded, y_test_encoded, 
                encoder, imputer, scaler, pca_obj)
    
    return X_train_imputed, X_test_imputed, y_train_encoded, y_test_encoded, encoder, imputer

In [9]:
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

def run_xgboost_experiment(X_train, y_train, X_test, y_test):
    model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', objective='multi:softprob', num_class=3)
    model.fit(X_train, y_train)
    y_prob = model.predict_proba(X_test)
    # Assuming you want to treat the problem as binary for ROC AUC purposes
    auc_score = roc_auc_score(y_test, y_prob, multi_class='ovo')  # One-vs-One strategy for multi-class ROC
    return auc_score

In [10]:
#RESNET

# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, BatchNormalization, ReLU, Add, Dropout
# from tensorflow.keras.models import Model
# from sklearn.metrics import roc_auc_score

# def build_resnet_model(input_shape, num_classes):
#     inputs = Input(shape=(input_shape,))
#     x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))(inputs)
#     x = BatchNormalization()(x)
#     x = ReLU()(x)

#     for _ in range(3):  # Increased depth
#         x_shortcut = x
#         x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
#         x = BatchNormalization()(x)
#         x = ReLU()(x)
#         x = Dense(64, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x)
#         x = BatchNormalization()(x)
#         x = Add()([x, x_shortcut])
#         x = ReLU()(x)

#     x = Dense(32)(x)
#     x = ReLU()(x)
#     x = Dropout(0.5)(x)  # Regularization to prevent overfitting
#     outputs = Dense(num_classes, activation='softmax')(x)
    
#     model = Model(inputs=inputs, outputs=outputs)
#     model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#     return model

# def train_predict_evaluate_resnet(X_train, y_train, X_test, y_test):
#     # Replace NaNs with a placeholder and use an indicator for missing values
#     for column in X_train.columns:
#         X_train[column + '_missing'] = X_train[column].isna().astype(int)
#         X_train[column].fillna(-999, inplace=True)  # Placeholder value
    
#     for column in X_test.columns:
#         X_test[column + '_missing'] = X_test[column].isna().astype(int)
#         X_test[column].fillna(-999, inplace=True)  # Placeholder value

#     num_features = X_train.shape[1]
#     num_classes = len(set(y_train))

#     # Build and train the ResNet model
#     model = build_resnet_model(num_features, num_classes)
#     model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)
    
#     # Predict probabilities on the test set and calculate AUC
#     y_prob = model.predict(X_test)
#     auc_score = roc_auc_score(y_test, y_prob, multi_class='ovo')

#     return auc_score


In [15]:
train_data

,Path,Age,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,...,Asymmetry,Sex_Female,Sex_Male,Sex_Unknown,Frontal/Lateral_Frontal,Frontal/Lateral_Lateral,AP/PA_AP,AP/PA_LL,AP/PA_PA,AP/PA_RL
1,CheXpert-v1.0-small/train/patient00002/study2/...,87,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,...,0.596201,1,0,0,1,0,1,0,0,0
5,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,0.270156,1,0,0,1,0,0,0,1,0
6,CheXpert-v1.0-small/train/patient00004/study1/...,20,1.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,1,0,0,0,1,0,0,0,0
7,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,0.225228,0,1,0,1,0,0,0,1,0
8,CheXpert-v1.0-small/train/patient00005/study1/...,33,1.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,...,NaN,0,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223403,CheXpert-v1.0-small/train/patient64533/study1/...,75,NaN,NaN,1.0,NaN,NaN,1.0,NaN,NaN,...,0.285247,0,1,0,1,0,1,0,0,0
223405,CheXpert-v1.0-small/train/patient64534/study1/...,63,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,0.267634,0,1,0,1,0,1,0,0,0
223407,CheXpert-v1.0-small/train/patient64536/study2/...,61,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,0.727078,1,0,0,1,0,1,0,0,0
223409,CheXpert-v1.0-small/train/patient64537/study2/...,59,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,...,0.368473,0,1,0,1,0,1,0,0,0


In [16]:
# --- Imports ---
import numpy as np
import pandas as pd
import os
from PIL import Image

import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# --- Image Loading Function ---
def load_images(image_paths, base_path='', img_size=(224, 224)):
    images = []
    for img_rel_path in image_paths:
        full_path = os.path.join(base_path, img_rel_path)
        try:
            img = Image.open(full_path).convert('L')  # Convert to grayscale
            img = img.resize(img_size)
            img_array = np.array(img) / 255.0  # Normalize [0, 1]
            img_array = np.expand_dims(img_array, axis=-1)  # Add channel dimension
            images.append(img_array)
        except Exception as e:
            print(f"Error loading {full_path}: {e}")
            images.append(np.zeros((*img_size, 1)))  # Blank image if error
    return np.array(images, dtype=np.float32)

# --- Numerical Features Loading ---
def load_tabular_features(df):
    tabular_features = df[['Age', 'sex_male', 'sex_female', 'sex_unknown'] +
                           [col for col in df.columns if col.startswith('corner')] +
                           [col for col in df.columns if col.startswith('hist')]]
    return tabular_features.values.astype(np.float32)

# --- Build Multi-Input Model ---
def build_multi_input_model(image_shape, tabular_shape):
    # Image branch
    image_input = Input(shape=image_shape)
    base_cnn = ResNet50(weights=None, include_top=False, input_tensor=image_input)
    x_img = GlobalAveragePooling2D()(base_cnn.output)

    # Tabular branch
    tabular_input = Input(shape=(tabular_shape,))
    x_tab = Dense(128, activation='relu')(tabular_input)
    x_tab = Dense(64, activation='relu')(x_tab)

    # Merge
    combined = concatenate([x_img, x_tab])
    z = Dense(64, activation='relu')(combined)
    output = Dense(1, activation='sigmoid')(z)

    model = Model(inputs=[image_input, tabular_input], outputs=output)
    return model

# --- MAIN PIPELINE ---
def main_pipeline(train_data, base_image_path):
    # 1. Load images
    print("Loading images...")
    images_array = load_images(train_data['Path'], base_path=base_image_path, img_size=(224, 224))
    print("Images loaded:", images_array.shape)

    # 2. Load tabular features
    print("Loading numerical features...")
    tabular_array = load_tabular_features(train_data)
    print("Numerical features loaded:", tabular_array.shape)

    # 3. Normalize tabular features
    scaler = StandardScaler()
    tabular_array = scaler.fit_transform(tabular_array)

    # 4. Load labels
    labels_array = train_data['Label'].values  # Make sure you have 'Label' column

    # 5. Train-test split
    X_img_train, X_img_val, X_tab_train, X_tab_val, y_train, y_val = train_test_split(
        images_array, tabular_array, labels_array, test_size=0.2, random_state=42, stratify=labels_array
    )

    # 6. Build the model
    print("Building model...")
    model = build_multi_input_model(image_shape=(224, 224, 1), tabular_shape=tabular_array.shape[1])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

    # 7. Train the model
    print("Training model...")
    model.fit([X_img_train, X_tab_train], y_train,
              validation_data=([X_img_val, X_tab_val], y_val),
              epochs=15, batch_size=32)

    return model


In [ ]:
model = main_pipeline(train_data, base_image_path='/path/to/images')

In [11]:
train_data.columns

Index(['Path', 'Age', 'No Finding', 'Enlarged Cardiomediastinum',
       'Cardiomegaly', 'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation',
       'Pneumonia',
       ...
       'Asymmetry', 'Sex_Female', 'Sex_Male', 'Sex_Unknown',
       'Frontal/Lateral_Frontal', 'Frontal/Lateral_Lateral', 'AP/PA_AP',
       'AP/PA_LL', 'AP/PA_PA', 'AP/PA_RL'],
      dtype='object', length=484)

# TRAIN AND TEST WITHOUT CLINICAL FEATURES

In [12]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
# train_data_no_clinical = train_data.drop(['Pleural Effusion', 'Path','Right_Lung_Intensity', 'Left_Lung_Intensity', 'Asymmetry', ], axis=1)
# feature_columns = train_data_no_clinical.columns  # Modify as needed
fixed_columns = ['Age', 'Sex_Male', 'Sex_Female', 'Sex_Unknown']

# Select columns that start with 'corner' or 'hist'
pattern_columns = [col for col in train_data.columns if col.startswith('corner') or col.startswith('hist')]

# Combine all desired columns
feature_columns = fixed_columns + pattern_columns
# Prepare the data
X_train, X_test, y_train, y_test, encoder, imputer = prepare_data(train_data, feature_columns, pca=False)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)


XGBoost AUC: 0.650409677457851


In [13]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

# TRAIN AND TEST WITH CLINICAL FEATURES

In [ ]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_clinical = train_data.drop(['Pleural Effusion', 'Path'], axis=1)
feature_columns = train_data_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=False)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

# For ResNet, ensure you have the appropriate setup or use another suitable model
# resnet_auc = run_resnet_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)

In [ ]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)

# PCA

In [ ]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_clinical = train_data.drop(['Pleural Effusion', 'Path'], axis=1)
feature_columns = train_data_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=True)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

# For ResNet, ensure you have the appropriate setup or use another suitable model
# resnet_auc = run_resnet_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)

In [ ]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)

In [ ]:
# Define the subset of columns for an experiment
# Correct way to drop multiple columns
train_data_no_clinical = train_data.drop(['Pleural Effusion', 'Path','Right_Lung_Intensity', 'Left_Lung_Intensity', 'Asymmetry', ], axis=1)
feature_columns = train_data_no_clinical.columns  # Modify as needed

# Prepare the data
X_train, X_test, y_train, y_test = prepare_data(train_data, feature_columns, pca=True)

# Run XGBoost experiment
xgb_auc = run_xgboost_experiment(X_train, y_train, X_test, y_test)

print("XGBoost AUC:", xgb_auc)
# print("ResNet AUC:", resnet_auc)


In [ ]:
auc_score = train_predict_evaluate_resnet(X_train, y_train, X_test, y_test)
print("ResNet Model AUC Score:", auc_score)